## Steps to Build an RNN
1. Import Libraries: Bring in the required libraries, torch, torch.nn and torch.optim.
2. Define the RNN Model: Create a class for your RNN model by, subclassing torch.nn.Module.
3. Preparing Data: Data must be in a sequential format in order for RNNs to function properly. Preprocessing procedures like tokenization for text data, and normalization for time series data are frequently involved in this.
4. DataLoader in PyTorch: PyTorch provides the DataLoader class to easily handle batching, shuffling, and loading data in parallel. This is crucial for efficient training of RNNs.
5. Train the Model: Use a loss function and an optimizer to train your model on your dataset. Training Loop When training an RNN, the data is iterated over several times, or epochs and the model weights are updated by the use of backpropagation through time (BPTT)
6. Evaluate the Model: Test your model to see how well it performs on unseen data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [139]:
# Create synthentic data
def generate_data(num_samples, seq_len):
    X = []
    Y = []
    for i in range(num_samples):
        x = np.linspace(i*2*np.pi, (i+1)*2*np.pi, seq_len+1)
        print(x)
        sine_wave = np.sin(x)
        print(sine_wave)
        X.append(x[1:])
        Y.append(sine_wave[1:])
    return np.array(X), np.array(Y)


In [ ]:
num_samples = 10
x, y = generate_data(num_samples, 50)

In [ ]:
fig, ax = plt.subplots()
for i in range(num_samples):
    ax.plot(x[i], y[i])
plt.show()


In [ ]:
# convert to pytorch tensors
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
print(x.shape)

In [143]:
#Define an RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out)
        return out

In [144]:
input_size = 1
hidden_size = 20
output_size = 1
model = SimpleRNN(input_size, hidden_size, output_size)

In [ ]:
print(model.parameters)
#train the model using Mean Squared Error (MSE) loss and the Adam optimizer.
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

# training loop
num_epochs = 1000
#for collecting losses
train_loss_arr = []
for epoch in range(num_epochs):
    model.train()
    outputs = model(x.unsqueeze(2))
    loss = criterion(outputs, y.unsqueeze(2))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #accumulate the loss
    loss = loss.item()
    #accumulate training losses
    train_loss_arr.append(loss)


In [ ]:
print(train_loss_arr)

In [ ]:
plt.plot(train_loss_arr)
plt.show()

In [ ]:
num_samples = 10
x_, _ = generate_data(num_samples, 50)
# make the prediction
model.eval()
with torch.no_grad():
    predictions = model(x_)

#print(x)


In [ ]:
#plt.plot(x_, predictions[0], label='Predicted')
for i in range(6,10):
    plt.plot(x_[i], predictions[0])
for j in range(5):
    plt.plot(x_[j], y[j])
plt.show()